In [13]:
from sklearn.pipeline import Pipeline as SklearnPipeline
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.naive_bayes import MultinomialNB
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split


### cargamos el dataset

In [2]:
dataset = pd.read_csv('../data_process/FinalDataTrainer.csv')

### Dividimos la data entre prueba y entrenamiento

In [4]:
# Dividir el dataset en entrenamiento (80%) y prueba (20%)
X = dataset['processed_text']  # Características
y = dataset['sentiment']       # Variable objetivo 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40)

### Convertimos los datos a valores numericos

In [11]:
# Definir el pipeline completo
pipeline = ImbPipeline([
    ('vectorizer', TfidfVectorizer(max_features=10000)),  # Vectorización
    ('oversample', SMOTE(sampling_strategy={
        'enojado': 5000,       # Sobremuestrear a 5,000
        'insatisfecho': 10000, # Sobremuestrear a 10,000
        'neutral': 10000       # Sobremuestrear a 10,000
    })),
    ('undersample', RandomUnderSampler(sampling_strategy={
        'contento': 10000,     # Submuestrear a 10,000
        'positivo': 10000,     # Submuestrear a 10,000
        'negativo': 10000      # Submuestrear a 10,000
    })),
    ('model', MultinomialNB())  # Modelo
])

In [10]:
print(y_train.value_counts())

sentiment
contento        20831
positivo        17293
negativo        14576
insatisfecho     8307
neutral          7168
enojado          3389
Name: count, dtype: int64


### ENTRENAR EL MODELO 

In [12]:
pipeline.fit(X_train, y_train)
# Predecir con los datos de prueba
y_pred = pipeline.predict(X_test)

In [14]:
# Realizar predicciones
# Evaluar el modelo
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred))

Accuracy: 0.4740931194455313

Reporte de clasificación:
              precision    recall  f1-score   support

    contento       0.65      0.84      0.73      5270
     enojado       0.32      0.22      0.26       815
insatisfecho       0.30      0.37      0.33      2009
    negativo       0.37      0.18      0.24      3607
     neutral       0.33      0.25      0.28      1847
    positivo       0.42      0.47      0.44      4343

    accuracy                           0.47     17891
   macro avg       0.40      0.39      0.38     17891
weighted avg       0.45      0.47      0.45     17891



### GUARDAR EL MODELO

In [8]:
# Guardar el vectorizador
joblib.dump(vectorizer, 'vectorizer.pkl')

# Guardar el modelo entrenado
joblib.dump(modelo, 'modelo_sentimiento.pkl')

print("Modelo y vectorizador guardados correctamente.")

Modelo y vectorizador guardados correctamente.
